### Завдання

1. Завантажте та прочитайте кожен CSV-файл як окремий DataFrame.

2. Очистіть дані, видаляючи будь-які рядки з пропущеними значеннями.

3. Визначте загальну суму покупок за кожною категорією продуктів.

4. Визначте суму покупок за кожною категорією продуктів для вікової категорії від 18 до 25 включно.

5. Визначте частку покупок за кожною категорією товарів від сумарних витрат для вікової категорії від 18 до 25 років.

6. Виберіть 3 категорії продуктів з найвищим відсотком витрат споживачами віком від 18 до 25 років.

In [39]:
#імпорт бібліотек
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, round, sum, lit

##### 1. Завантажте та прочитайте кожен CSV-файл як окремий DataFrame.

In [40]:
#Створення spark-сесії з назвою HW3
spark = SparkSession.builder.appName("HW3").getOrCreate()

products = spark.read.csv("datasets_hw3/products.csv", header=True)
orders = spark.read.csv("datasets_hw3/purchases.csv", header=True)
users = spark.read.csv("datasets_hw3/users.csv", header=True)

In [41]:
#Ознайомлення з даними
products.select("*").limit(3).show()
orders.select("*").limit(3).show()
users.select("*").limit(3).show()

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+-----------+-----+
|         1|   Product_1|     Beauty|  8.3|
|         2|   Product_2|       Home|  8.3|
|         3|   Product_3|Electronics|  9.2|
+----------+------------+-----------+-----+

+-----------+-------+----------+----------+--------+
|purchase_id|user_id|product_id|      date|quantity|
+-----------+-------+----------+----------+--------+
|          1|     52|         9|2022-01-01|       1|
|          2|     93|        37|2022-01-02|       8|
|          3|     15|        33|2022-01-03|       1|
+-----------+-------+----------+----------+--------+

+-------+------+---+-----------------+
|user_id|  name|age|            email|
+-------+------+---+-----------------+
|      1|User_1| 45|user1@example.com|
|      2|User_2| 48|user2@example.com|
|      3|User_3| 36|user3@example.com|
+-------+------+---+-----------------+



##### 2. Очистіть дані, видаляючи будь-які рядки з пропущеними значеннями.

Спочатку порахуємо загальну кількість записів у таблиців, щоб дізнатись, чи є пропущені значення.
- У таблиці products пропущених значень немає.
- У таблицях users та purchases порівняємо кількість записів до та після видалення пропусків.

In [42]:
df_products = products.agg(
    count("category").alias("count_category"),
    count("price").alias("count_price"),
    count("product_name").alias("count_product_name"),
    count("product_id").alias("product_id_cnt")
)

df_products.show()

#no Nulls

+--------------+-----------+------------------+--------------+
|count_category|count_price|count_product_name|product_id_cnt|
+--------------+-----------+------------------+--------------+
|            49|         49|                49|            49|
+--------------+-----------+------------------+--------------+



In [43]:
def users_count(table = users):
    """Функція для підрахунку кількості записів у таблиці users"""
    df_users = table.agg(
    count("name").alias("count_name"),
    count("age").alias("count_age"),
    count("email").alias("count_email"),
    count("user_id").alias("user_id_count")
    )

    df_users.show()

users_count()

+----------+---------+-----------+-------------+
|count_name|count_age|count_email|user_id_count|
+----------+---------+-----------+-------------+
|        98|       98|         99|          100|
+----------+---------+-----------+-------------+



In [44]:
users_cleaned = users.dropna()
users_count(users_cleaned)

+----------+---------+-----------+-------------+
|count_name|count_age|count_email|user_id_count|
+----------+---------+-----------+-------------+
|        95|       95|         95|           95|
+----------+---------+-----------+-------------+



In [45]:
def orders_count(table=orders):
    """Функція для підрахунку кількості записів у таблиці purchases (переіменувала на orders)"""
    df_orders = table.agg(
    count("user_id").alias("count_user_id"),
    count("product_id").alias("count_product_id"),
    count("date").alias("count_date"),
    count("quantity").alias("count_quantity"),
    count("purchase_id").alias("user_id_count")
    )

    df_orders.show()

orders_count()

+-------------+----------------+----------+--------------+-------------+
|count_user_id|count_product_id|count_date|count_quantity|user_id_count|
+-------------+----------------+----------+--------------+-------------+
|          198|             199|       199|           199|          200|
+-------------+----------------+----------+--------------+-------------+



In [46]:
orders_cleaned = orders.dropna()

orders_count(orders_cleaned)

+-------------+----------------+----------+--------------+-------------+
|count_user_id|count_product_id|count_date|count_quantity|user_id_count|
+-------------+----------------+----------+--------------+-------------+
|          195|             195|       195|           195|          195|
+-------------+----------------+----------+--------------+-------------+



##### 3. Визначте загальну суму покупок за кожною категорією продуктів.

- Створимо окремий датасет orders_tbl з таблиці orders, який міститиме тільки записи, потрібні для даного задання ("product_id", "date", "quantity").
- Об'єднуємо (join) таблиці products та orders_tbl.
- Створюємо новий атрибут total_price для обчислення загальної вартості products у замовленні. Для цього множимо price і quantity.
- Групуємо за атрибутом category.
- Рахуємо суми замовлень за кожною категорією.

In [47]:
orders_tbl = orders_cleaned.select("product_id", "date", "quantity")

combined_table = orders_tbl.join(products, orders_tbl.product_id==products.product_id, "left")\
            .drop(products.product_id)\
            .dropna(subset=["category"])\
            .withColumn("price", col("price").cast("float"))\
            .withColumn("quantity", col("quantity").cast("int"))\
            .withColumn("total_price", round(col("price")*(col("quantity")), 2))\
            .select("category", "total_price")\
            .groupBy("category")\
            .agg(round(sum("total_price"), 2).alias("revenue_by_category"))

combined_table.show()

+-----------+-------------------+
|   category|revenue_by_category|
+-----------+-------------------+
|       Home|             1552.2|
|     Sports|             1802.5|
|Electronics|             1174.8|
|   Clothing|              790.3|
|     Beauty|              459.9|
+-----------+-------------------+



##### 4. Визначте суму покупок за кожною категорією продуктів для вікової категорії від 18 до 25 включно.

In [48]:
users_1825 = users_cleaned.select("*")\
                        .withColumn("age", col("age").cast("int"))\
                        .where((col("age")>=18) & (col("age")<=25))
usr_orders = orders_cleaned.join(
        users_1825, orders_cleaned.user_id==users_1825.user_id, "inner")\
        .join(products, orders_cleaned.product_id==products.product_id, "left")\
        .withColumn("price", col("price").cast("float"))\
        .withColumn("quantity", col("quantity").cast("int"))\
        .withColumn("total_price", round(col("price")*(col("quantity")), 2))\
        .groupBy("age", "category")\
        .agg(round(sum("total_price"), 2).alias("sum_total_price")).dropna()

usr_orders.orderBy(col("age").asc(), col("sum_total_price").desc()).show()

+---+-----------+---------------+
|age|   category|sum_total_price|
+---+-----------+---------------+
| 18|       Home|          186.4|
| 18|Electronics|           73.6|
| 20|       Home|           84.6|
| 20|     Sports|           76.8|
| 20|Electronics|           46.0|
| 20|   Clothing|           36.0|
| 20|     Beauty|           33.2|
| 21|   Clothing|           21.0|
| 21|       Home|            8.8|
| 21|     Sports|            8.4|
| 23|Electronics|           84.6|
| 23|     Sports|           67.5|
| 23|       Home|           28.7|
| 24|       Home|           81.3|
| 24|   Clothing|           50.4|
| 24|     Sports|           33.0|
| 24|Electronics|           13.0|
| 25|   Clothing|          137.6|
| 25|     Sports|          124.8|
| 25|Electronics|           32.4|
+---+-----------+---------------+
only showing top 20 rows


##### 5. Визначте частку покупок за кожною категорією товарів від сумарних витрат для вікової категорії від 18 до 25 років.

In [51]:
total_spent = usr_orders.agg(sum("sum_total_price").alias("total_spent"))\
                    .collect()[0]["total_spent"]

users_1825_stats = usr_orders.groupBy("category") \
            .agg(round(sum("sum_total_price"), 2).alias("spent_by_price")) \
            .withColumn("total_spent", round(lit(total_spent), 2))\
            .withColumn("spent_ratio", 
            round(col("spent_by_price").cast("float") / col("total_spent").cast("float"), 2))

users_1825_stats.show()

+-----------+--------------+-----------+-----------+
|   category|spent_by_price|total_spent|spent_ratio|
+-----------+--------------+-----------+-----------+
|       Home|         389.8|     1236.3|       0.32|
|     Sports|         310.5|     1236.3|       0.25|
|Electronics|         249.6|     1236.3|        0.2|
|   Clothing|         245.0|     1236.3|        0.2|
|     Beauty|          41.4|     1236.3|       0.03|
+-----------+--------------+-----------+-----------+



##### 6. Виберіть 3 категорії продуктів з найвищим відсотком витрат споживачами віком від 18 до 25 років.

In [55]:
users_1825_stats.select("category", "spent_by_price", "total_spent", "spent_ratio")\
                .orderBy(col("spent_ratio").desc()).limit(3).show()

+-----------+--------------+-----------+-----------+
|   category|spent_by_price|total_spent|spent_ratio|
+-----------+--------------+-----------+-----------+
|       Home|         389.8|     1236.3|       0.32|
|     Sports|         310.5|     1236.3|       0.25|
|Electronics|         249.6|     1236.3|        0.2|
+-----------+--------------+-----------+-----------+



In [ ]:
spark.stop()